# * VIS Daily

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Input parameter '''

op_dir = 'data'
op_vis_file = f'vis_daily_{str_curr_dt}'

# v_year = 0
# v_month_start = 0
# v_month_end = 0
# v_date = 20250101

print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_vis_file: {op_vis_file}')
# print(f'\n   -> v_year: {v_year}')
# print(f'   -> v_month_start: {v_month_start}')
# print(f'   -> v_month_end: {v_month_end}')
# print(f'\n   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_vis_file: vis_daily_20260121


## Import Transaction
-> AGG_PERF_NEWCO

In [3]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()

# {v_start_date}

query = (f"""
    SELECT /*+PARALLEL(8)*/
        A.TM_KEY_YR, A.TM_KEY_QTR, A.TM_KEY_MTH, A.TM_KEY_WK, A.TM_KEY_DAY
        , PRODUCT_GRP, COMP_CD, METRIC_GRP--, METRIC_NAME_GROUP
        , METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
        , ACTUAL_AS_OF, ACTUAL_SNAP, TARGET_SNAP, ACH_SNAP
        , ACTUAL_AGG_WK, TARGET_AGG_WK, ACH_AGG_WK, WOW, WOW_PERCENT
        , ACTUAL_AGG_MTH, TARGET_AGG_MTH, ACH_AGG_MTH, MOM, MOM_PERCENT
        , ACTUAL_AGG_QTR, TARGET_AGG_QTR, ACH_AGG_QTR, QOQ, QOQ_PERCENT
        , ACTUAL_AGG_YR, TARGET_AGG_YR, ACH_AGG_YR--, YOY, YOY_PERCENT
        , MTD_YOY, MTD_YOY_PERCENT
        , BASELINE_AGG_YR, PPN_TM
        , T.DAYS_IN_MONTH, T.PERIODFLAG
        , O.AREA_CD_G, O.AREA_NAME_G

    FROM GEOSPCAPPO.AGG_PERF_NEWCO A

    LEFT JOIN GEOSPCAPPO.DIM_TIME_CITRINE T
        ON T.TM_KEY_DAY = A.TM_KEY_DAY
        
    LEFT JOIN (
        SELECT H.AREA_CD AS AREA_CD_H, G.AREA_CD AS AREA_CD_G, G.AREA_NAME AS AREA_NAME_G
        FROM GEOSPCAPPO.DIM_MOOC_AREA_CITRINE H
        LEFT JOIN (
            SELECT AREA_CD, AREA_NAME
            FROM GEOSPCAPPO.DIM_MOOC_AREA_CITRINE
            WHERE AREA_TYPE = 'G'
        ) G
            ON G.AREA_CD = H.AREA_CD_G
        WHERE H.AREA_TYPE = 'H'
    ) O
        ON O.AREA_CD_H = A.AREA_CD

    WHERE AREA_CD IN ('P', 'GX1', 'GX2', 'GX3', 'GX4', 'GX5', 'GX6', 'GX7', 'GX8')
    AND METRIC_CD IN (
        -->> Revenue
        'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        , 'B2R010100' --Postpaid Revenue B2C
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
        -->> Subs
    --	, 'B2S010602' --Postpaid SubBase B2C
    --	, 'DB2S010602' --Postpaid SubBase B2C : DTAC
    --	, 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'B1S000601' --Prepaid Active Caller 30D Rolling : Thai Mass
        , 'B1S000602' --Prepaid Active Caller 30D Rolling : AEC/Migrants
        , 'B1S000603' --Prepaid Active Caller 30D Rolling : Tourists
        , 'DB1S000600' --Prepaid Active Caller 30D Rolling : DTAC
        , 'DB1S000601' --Prepaid Active Caller 30D Rolling : Thai Mass : DTAC
        , 'DB1S000602' --Prepaid Active Caller 30D Rolling : AEC/Migrants : DTAC
        , 'DB1S000603' --Prepaid Active Caller 30D Rolling : Tourists : DTAC
        , 'TB1S000600' --Prepaid Active Caller 30D Rolling : TMH
        , 'TB1S000601' --Prepaid Active Caller 30D Rolling : Thai Mass : TMH
        , 'TB1S000602' --Prepaid Active Caller 30D Rolling : AEC/Migrants : TMH
        , 'TB1S000603' --Prepaid Active Caller 30D Rolling : Tourists : TMH
    --	, 'TB3S020604' --FTTx SubBase
        , 'TB3S000600' --FTTx Reported SubBase
        , 'TB4S000500' --TVS Active Subs
        , 'TNSC00150' --TrueID TV Box MAU
        -->> Churn
        , 'B2S010601' --Postpaid 60DPD B2C
        , 'DB2S010601' --Postpaid 60DPD B2C : DTAC
        , 'TB2S010601' --Postpaid 60DPD B2C : TMH
        , 'B2S010603' --Postpaid Gain/Loss 60DPD B2C ?
        , 'DB2S010603' --Postpaid Gain/Loss 60DPD B2C : DTAC
        , 'TB2S010603' --Postpaid Gain/Loss 60DPD B2C : TMH
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'DB2S010200' --Postpaid Churn Subs B2C : DTAC
        , 'DB2S010201' --Postpaid Churn Subs Voluntary B2C : DTAC
        , 'DB2S010202' --Postpaid Churn Subs Involuntary B2C : DTAC
        , 'TB2S010200' --Postpaid Churn Subs B2C : TMH
        , 'TB2S010201' --Postpaid Churn Subs Voluntary B2C : TMH
        , 'TB2S010202' --Postpaid Churn Subs Involuntary B2C : TMH
        , 'B1S000200' --Prepaid Churn Subs
        , 'DB1S000200' --Prepaid Churn Subs : DTAC
        , 'TB1S000200' --Prepaid Churn Subs : TMH
        , 'TB3S020603' --FTTx 60DPD
        , 'TB3S020606' --FTTx Gain/Loss 60DPD
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
    --	, 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
    --	, 'TSER14100' --TVS Churn Subs
    --	, 'TSER14102' --TVS Churn Subs Voluntary
    --	, 'TSER14103' --TVS Churn Subs Involuntary
        -->> Revenue ARPU
        , 'B7R010101' --Postpaid ARPU B2C
        , 'DB7R010101' --Postpaid ARPU B2C : DTAC
        , 'TB7R010101' --Postpaid ARPU B2C : TMH
        , 'B7R000102' --Prepaid ARPU
        , 'DB7R000102' --Prepaid ARPU : DTAC
        , 'TB7R000102' --Prepaid ARPU : TMH
        , 'TB7R000103' --TOL ARPU
        , 'TB7R000104' --TVS ARPU
        -->> Inflow
        , 'B0R00010001CS' --Total Inflow M1
        , 'DB0R00010001CS' --Total Inflow M1 : DTAC
        , 'TB0R00010001CS' --Total Inflow M1 : TRUE
        , 'B2R010500CS' --Postpaid Inflow M1 B2C
        , 'DB2R010500CS' --Postpaid Inflow M1 B2C : DTAC
        , 'TB2R010500CS' --Postpaid Inflow M1 B2C : TMH
        , 'B1R000900CS' --Prepaid Inflow M1
        , 'DB1R000900CS' --Prepaid Inflow M1 : DTAC
        , 'TB1R000900CS' --Prepaid Inflow M1 : TMH
        , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
        , 'TB3R000602CS' --TOL Inflow M1 Connected : DataService
        , 'TB4R001004CS' --TVS CMDU Inflow M1
        , 'TB4R001700CS' --TVS Now Inflow M1
        
        , 'B0R00010001CG' --Total Inflow M1 - GEO Channel
        , 'DB0R00010001CG' --Total Inflow M1 - GEO Channel : DTAC
        , 'TB0R00010001CG' --Total Inflow M1 - GEO Channel : TMH
        , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
        , 'DB2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel : DTAC
        , 'TB2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel : TMH
        , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
        , 'DB1R000900CG' --Prepaid Inflow M1 - GEO Channel : DTAC
        , 'TB1R000900CG' --Prepaid Inflow M1 - GEO Channel : TMH
        , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
        , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
        -->> GA
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'DB2S010100CS' --Postpaid Gross Adds B2C : DTAC
        , 'TB2S010100CS' --Postpaid Gross Adds B2C : TMH
        , 'B1S000101CS' --Prepaid Gross Adds
        , 'DB1S000101CS' --Prepaid Gross Adds : DTAC
        , 'TB1S000101CS' --Prepaid Gross Adds : TMH
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB3S000103CS' --TOL Gross Adds Connected : DataService
        , 'TB3S000104CS' --TOL Gross Adds Registration : Consumer
        , 'TB3S000105CS' --TOL Gross Adds Registration : DataService
        , 'TB4S001004CS' --TVS CMDU Gross Adds
        , 'TB4S001400CS' --TVS Now Gross Adds
        
        , 'B2S010100CG' --Postpaid Gross Adds B2C - GEO Channel
        , 'DB2S010100CG' --Postpaid Gross Adds B2C - GEO Channel : DTAC
        , 'TB2S010100CG' --Postpaid Gross Adds B2C - GEO Channel : TMH
        , 'B1S000101CG' --Prepaid Gross Adds - GEO Channel
        , 'DB1S000101CG' --Prepaid Gross Adds - GEO Channel : DTAC
        , 'TB1S000101CG' --Prepaid Gross Adds - GEO Channel : TMH
        , 'TB3S000104CG' --TOL Gross Adds Registration : Consumer - GEO Channel
        , 'TB3S000102CG' --TOL Gross Adds Connected : Consumer - GEO Channel
        , 'TB4S001400CG' --TVS Now Gross Adds - GEO Channel
        -->> GA PRPU/RC
        , 'B2R010600CS' --Postpaid GA RC B2C
        , 'DB2R010600CS' --Postpaid GA RC B2C : DTAC
        , 'TB2R010600CS' --Postpaid GA RC B2C : TMH
        , 'B1R001200CS' --Prepaid GA ARPU
        , 'DB1R001200CS' --Prepaid GA ARPU : DTAC
        , 'TB1R001200CS' --Prepaid GA ARPU : TMH
        , 'TB3R000800CS' --TOL GA ARPU
        , 'TB4R001800CS' --TVS Now GA ARPU
        
        , 'B2R010600CG' --Postpaid GA RC B2C - GEO Channel
        , 'DB2R010600CG' --Postpaid GA RC B2C : DTAC - GEO Channel
        , 'TB2R010600CG' --Postpaid GA RC B2C : TMH - GEO Channel
        , 'B1R001200CG' --Prepaid GA ARPU - GEO Channel
        , 'DB1R001200CG' --Prepaid GA ARPU : DTAC - GEO Channel
        , 'TB1R001200CG' --Prepaid GA ARPU : TMH - GEO Channel
        , 'TB3R000800CG' --TOL GA ARPU - GEO Channel
        , 'TB4R001800CG' --TVS Now GA ARPU - GEO Channel
        -->> Mobile MKS
        , 'VIN00011' --Mobile Subs Share : TMH & DTAC
        , 'VIN00012' --Mobile Subs Share : AIS
        , 'VIN00014' --Mobile Subs Share : DTAC
        , 'VIN00013' --Mobile Subs Share : TMH
        , 'VIN00037' --Postpaid Subs Share : TMH & DTAC
        , 'VIN00038' --Postpaid Subs Share : AIS
        , 'VIN00040' --Postpaid Subs Share : DTAC
        , 'VIN00039' --Postpaid Subs Share : TMH
        , 'VIN00029' --Prepaid Subs Share : TMH & DTAC
        , 'VIN00030' --Prepaid Subs Share : AIS
        , 'VIN00032' --Prepaid Subs Share : DTAC
        , 'VIN00031' --Prepaid Subs Share : TMH
    	, 'VIN00048' --Mobile Subs Share (Subs) : TMH & DTAC
    	, 'VIN00049' --Mobile Subs Share (Subs) : AIS
    	, 'VIN00051' --Mobile Subs Share (Subs) : DTAC
    	, 'VIN00050' --Mobile Subs Share (Subs) : TMH
    	, 'VIN00063' --Postpaid Subs Share (Subs) : TMH & DTAC
    	, 'VIN00064' --Postpaid Subs Share (Subs) : AIS
    	, 'VIN00066' --Postpaid Subs Share (Subs) : DTAC
    	, 'VIN00065' --Postpaid Subs Share (Subs) : TMH
    	, 'VIN00059' --Prepaid Subs Share (Subs) : TMH & DTAC
    	, 'VIN00060' --Prepaid Subs Share (Subs) : AIS
    	, 'VIN00062' --Prepaid Subs Share (Subs) : DTAC
    	, 'VIN00061' --Prepaid Subs Share (Subs) : TMH
        -->> Broadband MKS
        , 'VIN00080' --Broadband Subs Share : AIS & 3BB
        , 'VIN00081' --Broadband Subs Share : TOL
        , 'VIN00082' --Broadband Subs Share : 3BB
        , 'VIN00083' --Broadband Subs Share : AIS
        , 'VIN00084' --Broadband Subs Share : NT
    	, 'VIN00085' --Broadband Subs Share (Subs) : AIS & 3BB
    	, 'VIN00086' --Broadband Subs Share (Subs) : TOL
    	, 'VIN00087' --Broadband Subs Share (Subs) : 3BB
    	, 'VIN00088' --Broadband Subs Share (Subs) : AIS
    	, 'VIN00089' --Broadband Subs Share (Subs) : NT
        )
    AND A.TM_KEY_DAY >= {v_start_date}
""")


try:
    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\nDataFrame: {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_vis_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_vis_file}.csv" successfully')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2026-01-21, 10:42:23

DataFrame: 439285 rows, 43 columns

   -> Generate "vis_daily_20260121.csv" successfully


## Review

In [4]:
src_df

,TM_KEY_YR,TM_KEY_QTR,TM_KEY_MTH,TM_KEY_WK,TM_KEY_DAY,PRODUCT_GRP,COMP_CD,METRIC_GRP,METRIC_CD,METRIC_NAME,...,TARGET_AGG_YR,ACH_AGG_YR,MTD_YOY,MTD_YOY_PERCENT,BASELINE_AGG_YR,PPN_TM,DAYS_IN_MONTH,PERIODFLAG,AREA_CD_G,AREA_NAME_G
0,2025,20254,202511,2025046,20251114,Postpaid,ALL,Sales,B2R010600CG,Postpaid GA RC B2C - GEO Channel,...,NaN,NaN,59.652233,13.204864,NaN,2026-01-21 03:15:50,30,N,None,None
1,2025,20254,202511,2025046,20251115,Postpaid,ALL,Sales,B2R010600CG,Postpaid GA RC B2C - GEO Channel,...,NaN,NaN,61.248915,13.638763,NaN,2026-01-21 03:15:50,30,N,None,None
2,2025,20254,202511,2025047,20251120,Postpaid,ALL,Sales,B2R010600CG,Postpaid GA RC B2C - GEO Channel,...,NaN,NaN,63.184190,14.073847,NaN,2026-01-21 03:15:50,30,N,None,None
3,2025,20254,202511,2025047,20251121,Postpaid,ALL,Sales,B2R010600CG,Postpaid GA RC B2C - GEO Channel,...,NaN,NaN,66.635405,14.950281,NaN,2026-01-21 03:15:50,30,N,None,None
4,2025,20253,202508,2025032,20250810,Prepaid,DTAC,Subs,DB1S000602,Prepaid Active Caller 30D Rolling : AEC/Migran...,...,NaN,NaN,NaN,NaN,NaN,2026-01-21 03:15:50,31,N,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439280,2025,20253,202509,2025039,20250926,TVS,TRUE,Sales,TB4S001400CG,TVS Now Gross Adds - GEO Channel,...,70805.0,20.878469,733.000000,79.847495,NaN,2026-01-21 03:15:50,30,N,None,None
439281,2025,20254,202511,2025044,20251102,Prepaid,TRUE,Subs,TB1S000601,Prepaid Active Caller 30D Rolling : Thai Mass ...,...,NaN,NaN,NaN,NaN,NaN,2026-01-21 03:15:50,30,N,None,None
439282,2025,20254,202511,2025045,20251107,Prepaid,TRUE,Subs,TB1S000601,Prepaid Active Caller 30D Rolling : Thai Mass ...,...,NaN,NaN,NaN,NaN,NaN,2026-01-21 03:15:50,30,N,None,None
439283,2025,20254,202511,2025045,20251108,Prepaid,TRUE,Subs,TB1S000601,Prepaid Active Caller 30D Rolling : Thai Mass ...,...,NaN,NaN,NaN,NaN,NaN,2026-01-21 03:15:50,30,N,None,None
